In [2]:
import numpy as np
import ctypes
import threading
import queue
import time
import json
from tqdm import tqdm

buffer = queue.Queue(10)

In [3]:
def producer(epoches, length, distribution="random"):
    for i in range(epoches):
        if distribution == "random":
            data = np.random.randint(0, (1 << 31) - 1, length)
        elif distribution == "sorted":
            data = np.arange(0, length)
        else:
            mu = length / 2
            sigma = length / 6
            samples = np.random.normal(mu, sigma, length)
            samples = np.clip(samples, 0, 100)
            data = np.round(samples).astype(int)
        buffer.put(data)

In [4]:
def record_time(method, distribution, array_range, epoches):
    record = {}
    for length in array_range:
        producer_thread = threading.Thread(
            target=producer, args=(epoches, length, distribution)
        )
        producer_thread.start()
        average_time = 0
        for _ in tqdm(
            range(epoches),
            desc=f"Sorting {distribution} {length:.2e} array by {method[0]}",
        ):
            arr = buffer.get()
            if method[0] == "numpy":
                start = time.time()
                method[1].sort(arr)
                average_time += time.time() - start
            else:
                array_ptr = arr.ctypes.data_as(ctypes.POINTER(ctypes.c_int32))
                start = time.time()
                method[1].sort(array_ptr, 0, int(length - 1))
                average_time += time.time() - start
        average_time /= epoches
        record[int(length)] = average_time
        with open(f"data/{method[0]} in {distribution}.json", "w") as f:
            json.dump(record, f)

In [6]:
methods = {
    "random_quick_sort": ctypes.CDLL("libs/random_quick_sort.so", winmode=0),
    "3ways_quick_sort": ctypes.CDLL("libs/3ways_quick_sort.so", winmode=0),
    "numpy": np,
    "naive_quick_sort": ctypes.CDLL("libs/naive_quick_sort.so", winmode=0),
}
distribution_types = ["random", "normal", "sorted"]
begin = 5
end = 9
samples = 20
array_range = np.logspace(begin, end, samples * (end - begin) + 1).astype(np.int32)
epoches = 10

for distribution in distribution_types:
     for method in methods.items():
         record_time(method, distribution, array_range, epoches)